#A Machine Learning study of High Achievers at the Open University using the Learning Analytics Data Set

##Introduction

    The Open University is a fantastic public service. It is designed for those who wish to study a degree but whose cirumstances do not allow for the typical on-campus school 
    leaver experience. Many students are significantly older than typical undergraduates, and many study online and part-time, receiving a degree in six to eight years.

    In 2017 the University made public a large dataset containing 


##
A lot of data on unequal outcomes focusses on average scores (including Nottingham’s own dashboards). Work with the database schema itself has mainly focussed on machine learning applications to the Virtual Learning Environment. However, the database has a lot to offer beyond this. I want to focus just on the highest achievers and, specifically, whether having a previous degree makes a substantial difference to the probability of achieving highly. I have focussed on:
-	Those who received a distinction in the module the first time they took it.
-	The top ten scorers in the tutor-marked and computer-marked portions of a module.

At the advice of the Open University itself, I analyse the February and October presentations separately as their content and structure often differ from each other.


##Database Creation
    First, I created the necessary tables in SQL. I mostly use the recommendations on the University website, although sometimes these do not make total sense, so I use my own judgement.

##Data Load
	Next, I loaded the data into SQL

##Data Transformation
	As the data is presented over seven tables, I had to transform the learning engagement clicks data into a usable format for XGBoost. I used this loop to get the sum of clicks of each type of engagement. Not all of these types of engagement are present in every module. Later on, in python, I will automatically filter this inforamtion such that only resources available for the module in question are considered as features. 